# Перешитов Кирилл Александрович

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
from sklearn.model_selection import train_test_split
import scipy.stats as stats
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics
import os

df = pd.read_csv('/kaggle/input/ramen-ratings/ramen-ratings.csv')
df.head()



# EDA

In [ ]:
df.shape #

In [ ]:
df.describe()

In [ ]:
df = df.drop('Top Ten',axis=1) #Дропаем топ-10 т.к. там много пропущенных значений

In [ ]:
df.isnull().sum() # Найдем пропущенные значения

In [ ]:
df[df['Style'].isnull()]

In [ ]:
# Заполняем пропуски в поле Style
df.loc[2442,'Style'] = 'Bowl'  
df.loc[2152,'Style'] = 'Pack'  

In [ ]:
df['Variety'].value_counts().head(10).plot(kind='bar')

In [ ]:
df['Stars'].value_counts()

In [ ]:
# Есть 3 Unrated позиции. Сделаем функцию по заменее Unrated на 0
def unrated_to_zero(df):
    df.loc[df['Stars'] == 'Unrated'] = 0 
    return df

unrated_to_zero(df)
df['Stars'].value_counts()

In [ ]:
# Переведем категорию Style в цифровое значение

df = df.drop([32,122,993], axis = 0)
df['Style'].value_counts()


In [ ]:
from sklearn import preprocessing
data = df.apply(preprocessing.LabelEncoder().fit_transform)
data.head()

In [ ]:
data['Style'].value_counts()

In [ ]:
data['Category'] = [0 if x == 5 else 1 for x in data['Style']] 
  
# Print the DataFrame 
print(data.head()) 

Разбиваем модель на тестовую и обучающую выборку

In [ ]:
from sklearn.model_selection import train_test_split
df=data.drop('Style',axis= 1)
X=df.drop('Category',axis=1)
y=df['Category']

dfx=X
dfy=y

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=425)

# Логистическая регрессия

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train,y_train)

In [ ]:
y_pred_lr = classifier.predict(X_test)
cm = confusion_matrix(y_true=y_test,y_pred=y_pred_lr)

In [ ]:
# Фунцкия для вывода на печать всех метрик модели
def LR_report():
    print('\033[1m' +'Logistic Regression report')
    print('\033[0m')
    print('Confusion Matrix for LR\n',cm)
    print('')
    print('Accuracy score:', accuracy_score(y_test,y_pred_lr))
    print('')
    print('Classification report:')
    print(classification_report(y_test,y_pred_lr))
    print('')
    print('')
LR_report()


# Метод k-ближайших соседей

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors= 12)
classifier.fit(X_train,y_train)

y_pred_k = classifier.predict(X_test)
cm_kn = confusion_matrix(y_true=y_test,y_pred=y_pred_k)
acc_sc_kn = accuracy_score(y_test,y_pred_k)
pr_sc_kn = metrics.precision_score(y_test,y_pred_k,average='macro')
recall_kn = metrics.recall_score(y_test,y_pred_k,average='macro')

In [ ]:
def k_neigh_report():
    print('\033[1m' + 'K Neighbors Classifier report')
    print('\033[0m')
    print('Confusion Matrix \n',cm_kn)
    print('')
    print('Accuracy score:', acc_sc_kn)
    print('')
    print('Precision score:', pr_sc_kn)
    print('')
    print('Recall score:', recall_kn)
    print('')
    print('')
    
k_neigh_report()

# Наивный байесовский классификатор

In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train,y_train)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
cm_nb = confusion_matrix(y_true=y_test,y_pred=y_pred)


def naive_bayes_report():
    print('\033[1m' +'Naive Bayes report')
    print ('\033[0m')
    print('Confusion Matrix \n',cm_nb)
    print('')
    print('Accuracy score:', accuracy_score(y_test,y_pred))
    print('')
    print('Precision score:', metrics.precision_score(y_test,y_pred,average='macro'))
    print('')
    print('Recall score:', metrics.recall_score(y_test,y_pred,average='macro'))
    print('')
    print('')
naive_bayes_report()


# Функция для сравнения перфоманса всех моделей

In [ ]:
def reports():
    LR_report()
    k_neigh_report()
    naive_bayes_report()
    
reports()